In [1]:
"Dataset_04.csv"

<img src="Dataset_04.png" height="350" width="500">

### 1. 한국인의 1 인당 육류 소비량이 해가 갈수록 증가하는 것으로 보여 상관분석을 통하여 확인하려고 한다 .
- 데이터 파일로부터 한국 데이터만 추출한다. 한국은 KOR 로 표기되어 있다
- 년도별 육류 소비량 합계를 구하여 TIME 과 Value 간의 상관분석을 수행하고 상관계수를 소수점 셋째 자리에서 반올림하여 소수점 둘째 자리까지만 기술하시오.

(답안 예시) 0.55

In [2]:
import pandas as pd
from scipy.stats import pearsonr

In [3]:
df = pd.read_csv("Dataset_04.csv")
df.head(2)

,LOCATION,SUBJECT,TIME,Value
0,AUS,BEEF,1991,27.721815
1,AUS,BEEF,1992,26.199591


In [4]:
df_condi = df.loc[(df["LOCATION"] == "KOR")]
df_condi

,LOCATION,SUBJECT,TIME,Value
442,KOR,BEEF,1991,5.085481
443,KOR,BEEF,1992,5.685434
444,KOR,BEEF,1993,5.325765
445,KOR,BEEF,1994,5.698819
446,KOR,BEEF,1995,6.568429
...,...,...,...,...
583,KOR,SHEEP,2022,0.197951
584,KOR,SHEEP,2023,0.199098
585,KOR,SHEEP,2024,0.200266
586,KOR,SHEEP,2025,0.201451


In [13]:
stat, p = pearsonr(df_condi["TIME"], df_condi["Value"])

In [15]:
print(stat)
print(p)

0.32671137230310565
5.7035818920684335e-05


-------------------------------------------------------------

In [5]:
df_kor = df.loc[df["LOCATION"] == "KOR",]

In [27]:
df_kor["SUBJECT"].unique()

array(['BEEF', 'PIG', 'POULTRY', 'SHEEP'], dtype=object)

In [6]:
df_kor_agg = df_kor.groupby("TIME")["Value"].sum().reset_index()

In [7]:
df_kor.corr()

,TIME,Value
TIME,1.000000,0.326711
Value,0.326711,1.000000


In [29]:
df_kor_agg.corr().round(2)

,TIME,Value
TIME,1.00,0.96
Value,0.96,1.00


### 2. 한국 인근 국가 가운데 식생의 유사성이 상대적으로 높은 일본(JPN) 과 비교하여 , 연도별 소비량에 평균 차이가 있는지 분석하고자 한다.
- 두 국가의 데이터중 공통으로 기록된 연도를 기준으로 분석을 실시하시오.
- 두 국가의 육류별 소비량을 연도기준으로 비교하는 대응표본 t 검정을 수행하시오.
- 두 국가 간의 연도별 소비량 차이가 없는 것으로 판단할 수 있는 육류 종류를 모두 적으시오. 
- 가설검정은 유의수준 5% 기준으로 실시하시오.
(알파벳 순서) (답안 예시) BEEF, PIG, POULTRY, SHEEP

------------------------------------------------------------

In [32]:
df_jpn=df.loc[df["LOCATION"] == "JPN", ]

In [34]:
df_join = df_kor.merge(df_jpn,
                       left_on = ["SUBJECT", "TIME"],
                       right_on = ["SUBJECT", "TIME"],
                       how = "inner")

In [35]:
print(len(df_join))
df_join.head(3)

146


,LOCATION_x,SUBJECT,TIME,Value_x,LOCATION_y,Value_y
0,KOR,BEEF,1991,5.085481,JPN,6.517055
1,KOR,BEEF,1992,5.685434,JPN,6.766244
2,KOR,BEEF,1993,5.325765,JPN,7.406407


In [36]:
from scipy.stats import ttest_rel

In [39]:
stat, p = ttest_rel(df_join.loc[df_join["SUBJECT"] == "BEEF", "Value_x"],
                    df_join.loc[df_join["SUBJECT"] == "BEEF", "Value_y"])
round(p, 3) # 귀무가설을 기각하고 대립가설을 채택하여 두 집단간 평균 차이가 유의미하게 남

0.0

In [42]:
stat, p = ttest_rel(df_join.loc[df_join["SUBJECT"] == "PIG", "Value_x"],
                    df_join.loc[df_join["SUBJECT"] == "PIG", "Value_y"])
round(p, 3) # 귀무가설을 기각하고 대립가설을 채택하여 두 집단간 평균 차이가 유의미하게 남

0.0

In [41]:
stat, p = ttest_rel(df_join.loc[df_join["SUBJECT"] == "POULTRY", "Value_x"],
                    df_join.loc[df_join["SUBJECT"] == "POULTRY", "Value_y"])
round(p, 3) # 귀무가설을 기각하지 못하여 두 집단간 평균이 유의미하게 차이난다고 보기가 어려움

0.469

In [40]:
stat, p = ttest_rel(df_join.loc[df_join["SUBJECT"] == "SHEEP", "Value_x"],
                    df_join.loc[df_join["SUBJECT"] == "SHEEP", "Value_y"])
round(p, 3)# 귀무가설을 기각하고 대립가설을 채택하여 두 집단간 평균 차이가 유의미하게 남

0.0

### 3. (한국만포함한데이터에서) Time을 독립변수로, Value를 종속변수로하여 육류종류(SUBJECT) 별로 회귀분석을 수행하였을 때, 가장높은결정계수를가진모델의 학습 오차중 RMSE를 반올림 하여 소수점 둘 째자리까지 기술하시오. 
 - statsmodels의 ols() 함수를 사용하시오.
(답안예시) 21.12

In [43]:
from statsmodels.formula.api import ols
from sklearn.metrics import mean_squared_error


In [46]:
model = ols(formula = "Value ~ TIME",
            data = df_kor.loc[df_kor["SUBJECT"] == "BEEF", ]).fit()
model.rsquared

0.8354558055303123

In [51]:
model2 = ols(formula = "Value ~ TIME",
            data = df_kor.loc[df_kor["SUBJECT"] == "PIG", ]).fit()
model.rsquared

0.004082992795654161

In [52]:
model3 = ols(formula = "Value ~ TIME",
            data = df_kor.loc[df_kor["SUBJECT"] == "POULTRY", ]).fit()
model.rsquared

0.004082992795654161

In [53]:
model4 = ols(formula = "Value ~ TIME",
            data = df_kor.loc[df_kor["SUBJECT"] == "SHEEP", ]).fit()
model.rsquared

0.004082992795654161

In [54]:
pred = model3.predict(df_kor)
mean_squared_error(y_true = df_kor["Value"], 
                   y_pred = pred)**0.5

8.576805720117543